In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import os

from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model

# Helper function to load and preprocess images
def load_and_process_image(image_path, max_dim=512):
    img = Image.open(image_path)
    long = max(img.size)
    scale = max_dim / long
    img = img.resize(
        (round(img.size[0] * scale), round(img.size[1] * scale)), Image.Resampling.LANCZOS
    )
    img = np.array(img)
    img = np.expand_dims(img, axis=0)
    return vgg19.preprocess_input(img)

# Helper function to deprocess images
def deprocess_image(processed_img):
    x = processed_img.copy()
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]  # BGR to RGB
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load content and style images
content_image_path = 'content.jpg'  # Replace with your content image path
style_image_path = 'style.jpg'  # Replace with your style image path
content_image = load_and_process_image(content_image_path)
style_image = load_and_process_image(style_image_path)

# Display content and style images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Content Image")
plt.imshow(deprocess_image(content_image[0]))
plt.subplot(1, 2, 2)
plt.title("Style Image")
plt.imshow(deprocess_image(style_image[0]))
plt.show()

# Build the VGG19 model for feature extraction
vgg = vgg19.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

# Extract features from selected layers
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
output_layers = style_layers + content_layers
model = Model([vgg.input], [vgg.get_layer(layer).output for layer in output_layers])

# Define the loss functions
style_weight = 1e-2
content_weight = 1e4

def gram_matrix(tensor):
    x = tf.squeeze(tensor)
    x = tf.reshape(x, [-1, x.shape[-1]])
    n = tf.shape(x)[0]
    gram = tf.matmul(x, x, transpose_a=True) / tf.cast(n, tf.float32)
    return gram

def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
    style_weight, content_weight = loss_weights
    model_outputs = model(init_image)
    style_output_features = model_outputs[:len(style_layers)]
    content_output_features = model_outputs[len(style_layers):]

    style_score = 0
    content_score = 0

    # Calculate style loss
    for target, comb in zip(gram_style_features, style_output_features):
        style_score += tf.reduce_mean(tf.square(target - gram_matrix(comb)))

    # Calculate content loss
    for target, comb in zip(content_features, content_output_features):
        content_score += tf.reduce_mean(tf.square(target - comb))

    style_score *= style_weight
    content_score *= content_weight

    total_loss = style_score + content_score
    return total_loss

# Prepare the style and content targets
style_features = model(style_image)[:len(style_layers)]
content_features = model(content_image)[len(style_layers):]
gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

# Set up optimization
init_image = tf.Variable(content_image, dtype=tf.float32)
opt = tf.optimizers.Adam(learning_rate=5.0)

# Run the style transfer
epochs = 1000
for i in range(epochs):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, (style_weight, content_weight), init_image, gram_style_features, content_features)
    grads = tape.gradient(loss, init_image)
    opt.apply_gradients([(grads, init_image)])
    if i % 100 == 0:
        print(f"Step {i}, Loss: {loss.numpy()}")

# Display the final result
output_image = deprocess_image(init_image.numpy()[0])
plt.figure(figsize=(10, 10))
plt.title("Styled Image")
plt.imshow(output_image)
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ganes\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ganes\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ganes\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ganes\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\ganes\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ganes\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ganes\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ganes\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ganes\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\ganes\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ganes\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ganes\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ganes\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ganes\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\ganes\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle